<a href="https://colab.research.google.com/github/Alexsumt97/ML_Framework/blob/master/Final_Gdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import csv
from collections import OrderedDict
import math
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import ast
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(list(code.values()))

['Make public statement', 'Appeal', 'Express intent to cooperate', 'Consult', 'Engage in diplomatic cooperation', 'Engage in material cooperation', 'Provide aid', 'Yield', 'Investigate', 'Demand', 'Disapprove', 'Reject', 'Threaten', 'Protest', 'Exhibit military posture', 'Reduce relations', 'Coerce', 'Assault', 'Fight', 'Use unconventional mass violence']


# City

In [ ]:
#@title Default title text
code = {
        "1":"Make public statement",
        "2":"Appeal",
        "3":"Express intent to cooperate",
        "4":"Consult",
        "5":"Engage in diplomatic cooperation",
        "6":"Engage in material cooperation",
        "7":"Provide aid",
        "8":"Yield",
        "9":"Investigate",
        "10":"Demand",
        "11":"Disapprove",
        "12":"Reject",
        "13":"Threaten",
        "14":"Protest",
        "15":"Exhibit military posture",
        "16":"Reduce relations",
        "17":"Coerce",
        "18":"Assault",
        "19":"Fight",
        "20":"Use unconventional mass violence"
      }
mapping = {
        "Make public statement":"1",
        "Appeal":"1",
        "Express intent to cooperate":"1",
        "Consult":"1",
        "Engage in diplomatic cooperation":"1",
        "Engage in material cooperation":"1",
        "Provide aid":"1",
        "Yield":"1",
        "Investigate":"-1",
        "Demand":"-1",
        "Disapprove":"-1",
        "Reject":"-1",
        "Threaten":"-1",
        "Protest":"-1",
        "Exhibit military posture":"-1",
        "Reduce relations":"-1",
        "Coerce":"-1",
        "Assault":"-1",
        "Fight":"-1",
        "Use unconventional mass violence":"-1"
      }


subdomain = {
    '1':'geopolitical_opportunity',
    '-1':'geopolitical_threat'
}


desc_mapping = {
        "Make public statement":"All public statements expressed verbally or in action",
        "Appeal":"All requests, proposals, suggestions and appeals",
        "Express intent to cooperate":"Offer, promise, agree to, or otherwise indicate willingness or commitment tocooperate",
        "Consult":"All consultations and meetings ",
        "Engage in diplomatic cooperation":"Initiate, resume, improve, or expand diplomatic, non-material cooperation",
        "Engage in material cooperation":"Initiate, resume, improve, or expand material cooperation or exchange",
        "Provide aid":"All provisions, extension of material aid",
        "Yield":"All yieldings, concessions",
        "Investigate":"All non-covert investigations",
        "Demand":"All demands and orders",
        "Disapprove":"Express disapprovals, objections, and complaints",
        "Reject":"All rejections and refusals",
        "Threaten":"All threats, coercive or forceful warnings with serious potential repercussions",
        "Protest":"All  civilian  demonstrations  and  other  collective  actions  carried  out  asprotests against the target actor",
        "Exhibit military posture":"All  military  or  police  moves  that  fall  short  of  the  actual  use  of  force",
        "Reduce relations":"All  reductions  in  normal,  routine,  or  cooperative  relations",
        "Coerce":"Repression, violence against civilians, or their rights or properties",
        "Assault":"Use of unconventional forms of violence which do not require high levels of organization or conventional weaponry",
        "Fight":"All uses of conventional force and acts of war typically by organized armed groups",
        "Use unconventional mass violence":"All uses of unconventional force that are meant to cause mass destruction,casualties, and suffering"
      }

In [ ]:
#@title city jsons
data=pd.read_csv('/content/drive/MyDrive/all_data/Data/GDELT/gdelt_all_final_11.csv')
data.drop(['Unnamed: 0','lat','lon'],axis=1,inplace=True)
data['viz_type'] = 'city level'
data = data.groupby(['name','Actor1CountryCode','EventRootCode','country','state','city','viz_type']).agg('sum').reset_index()
data['variable'] = data['EventRootCode'].astype(str).replace(list(code.keys()), list(code.values()))

data['direction'] = data['variable'].replace(list(mapping.keys()), list(mapping.values()))
data.drop(['EventRootCode','NumArticles','country'],axis=1,inplace=True)
data['value'] = ""
data['risk'] = ""
data = data.dropna()
final = pd.DataFrame(columns=list(data.columns))

countries = list(data['Actor1CountryCode'].unique())
for country in countries:
  d1 = data[data['Actor1CountryCode']==country]
  states = list(d1['state'].unique())
  for state in states:
    d2 = d1[d1['state']==state]
    cities = list(d2['city'].unique())
    for city in cities:
      d3 = d2[d2['city']==city]
      total = d3['NumMentions'].sum()
      d3['value'] = d3['NumMentions']*100/total
      scaler = MinMaxScaler(feature_range=(1, 5))
      value=np.array(d3['value'])
      sc = scaler.fit_transform(value.reshape(-1,1))
      sc= pd.Series(sc.flatten())
      sc1=[]
      for i in sc:
        sc1.append(round(i))
      d3['risk'] = sc1
      final = pd.concat([final,d3])
final.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_1.csv")    
final['coordinates'] = final['lat'].astype(str)+", "+final['lon'].astype(str)

final.drop(['lat','lon'],axis=1,inplace=True)

final.rename(columns={'Actor1CountryCode':'countrycode','name':'country'},inplace=True)
cor = list(final['coordinates'])
k = []
df = final
for c in cor:
  a = []
  c = c.split(',')
  a.append(float(c[0]))
  a.append(float(c[1]))
  k.append(a)
df['cor'] = k
df.drop(['coordinates'],axis=1,inplace=True)
df.rename(columns={'cor':'coordinates'},inplace=True)
df['ingested_time']  = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
df['latest'] = 'Yes'
df['transformed'] = 'Yes'
df['sub_domain'] = df['direction'].replace([1,-1], ['geopolitical_opportunity','geopolitical_threat'])
df['domain'] = 'geopolitics'
df['indicator'] = "Number of mentions"
df.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2.csv")

In [ ]:
final = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2.csv")
sl = pd.read_csv("/content/drive/MyDrive/all_data/Data/gdelt-city-latlon.csv")
final.drop('Unnamed: 0',axis=1,inplace=True)
sl.drop('Unnamed: 0',axis=1,inplace=True)
fd = pd.merge(final,sl,how='left',on=['country','state','city'])
fd = fd.dropna()
fd['coordinates'] = fd['latitude'].astype(str)+","+fd['longitude'].astype(str)
cor = list(fd['coordinates'])
k = []

for c in cor:
  a = []
  c = c.split(',')
  a.append(float(c[0]))
  a.append(float(c[1]))
  k.append(a)
fd['cor'] = k
fd.drop(['coordinates','latitude','longitude'],axis=1,inplace=True)
fd.rename(columns={'cor':'coordinates'},inplace=True)

In [ ]:
fd = fd.dropna()
fd.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2.csv")

In [ ]:
final = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2.csv")
final.drop('Unnamed: 0',axis=1,inplace=True)
f = pd.DataFrame(columns=list(final.columns))
countries = list(final['country'].unique())
for country in countries:
  d1 = final[final['country']==country]
  states = list(d1['state'].unique())
  for state in states:
    d2 = d1[d1['state']==state]
    cities = list(d2['city'].unique())
    for city in cities:
      d3 = d2[d2['city'] == city]
      d3 = d3.sort_values(by=['risk'], ascending=False)
      d3 = d3[d3['risk']>3]
      d3 = d3.head(3)
      f = pd.concat([f,d3])
  
f['data'] = "top trends"
f.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2_top_trends.csv")

In [ ]:
f = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2.csv")
f['description'] = f['variable'].astype(str).replace(list(desc_mapping.keys()), list(desc_mapping.values()))
f['frequency'] = "Annual"
f.drop("Unnamed: 0",axis=1,inplace=True)

,Unnamed: 0,country,countrycode,state,city,viz_type,NumMentions,NumSources,variable,direction,value,risk,ingested_time,latest,transformed,sub_domain,domain,indicator,coordinates
0,0,Afghanistan,AFG,Badakhshan,Ashkasham,city level,2,2,Make public statement,1,9.090909,1,2021-03-12T19:42:46Z,Yes,Yes,1,geopolitics,Number of mentions,"[36.7090116, 71.5767799]"
1,1,Afghanistan,AFG,Badakhshan,Ashkasham,city level,4,1,Engage in diplomatic cooperation,1,18.181818,2,2021-03-12T19:42:46Z,Yes,Yes,1,geopolitics,Number of mentions,"[36.7090116, 71.5767799]"
2,2,Afghanistan,AFG,Badakhshan,Ashkasham,city level,16,4,Fight,-1,72.727273,5,2021-03-12T19:42:46Z,Yes,Yes,-1,geopolitics,Number of mentions,"[36.7090116, 71.5767799]"
3,3,Afghanistan,AFG,Badakhshan,Baharak,city level,61,16,Make public statement,1,9.854604,2,2021-03-12T19:42:46Z,Yes,Yes,1,geopolitics,Number of mentions,"[37.029015, 70.901202]"
4,4,Afghanistan,AFG,Badakhshan,Baharak,city level,12,2,Appeal,1,1.938611,1,2021-03-12T19:42:46Z,Yes,Yes,1,geopolitics,Number of mentions,"[37.029015, 70.901202]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557369,614226,Zimbabwe,ZWE,Midlands,Shurugwi,city level,4,2,Reject,-1,0.618238,1,2021-03-12T19:42:46Z,Yes,Yes,-1,geopolitics,Number of mentions,"[-19.75423355, 30.156915030014392]"
557370,614227,Zimbabwe,ZWE,Midlands,Shurugwi,city level,2,1,Protest,-1,0.309119,1,2021-03-12T19:42:46Z,Yes,Yes,-1,geopolitics,Number of mentions,"[-19.75423355, 30.156915030014392]"
557371,614228,Zimbabwe,ZWE,Midlands,Shurugwi,city level,62,15,Coerce,-1,9.582689,3,2021-03-12T19:42:46Z,Yes,Yes,-1,geopolitics,Number of mentions,"[-19.75423355, 30.156915030014392]"
557372,614229,Zimbabwe,ZWE,Midlands,Shurugwi,city level,15,3,Assault,-1,2.318393,1,2021-03-12T19:42:46Z,Yes,Yes,-1,geopolitics,Number of mentions,"[-19.75423355, 30.156915030014392]"


In [ ]:
#@title city level json
import csv, json
import ast 
li = []
with open("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    c = 0
    for _,country, countrycode, state, city, viz_type, NumMentions, NumSources, variable, direction,value,mean, ingested_time,latest,transformed,sub_domain,domain,indicator,coordinates,description,frequency  in reader:
        if c==0:
          c = 1
        else:
          li.append({
                "country"  : country,
                "city"     : city,
                "state"    : state,
                "NumMentions(Normalized)" : value,
                "variable"   : variable,
                "indicator"  : indicator,
                "countrycode"  : countrycode,
                "viz_type"  : viz_type,
                "NumMentions"  : NumMentions,
                "NumSources":NumSources,
                "coordinates": ast.literal_eval(coordinates),
                "direction":direction,
                "domain":domain,
                "sub_domain":sub_domain,
                "ingested_time": ingested_time,
                "latest":latest,
                "transformed": transformed,
                "value": mean,
                "description":description,
                "frequency":frequency


            }   
        )
        



In [ ]:
#@title save city json
with open('/content/drive/MyDrive/all_data/Data/GDELT/JSONS/gdelt_city_json.json', 'w') as json_file:
    json.dump(li, json_file)

In [ ]:
#@title city level json
import csv, json
import ast 
li = []
with open('/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_2_top_trends.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    c = 0
    for _,country, countrycode, state, city, viz_type, NumMentions, NumSources, variable, direction,value,mean, ingested_time,latest,transformed,sub_domain,domain,indicator,coordinates,description,frequency,data  in reader:
        if c==0:
          c = 1
        else:
          li.append({
                "country"  : country,
                "city"     : city,
                "state"    : state,
                "NumMentions(Normalized)" : value,
                "variable"   : variable,
                "indicator"  : indicator,
                "countrycode"  : countrycode,
                "viz_type"  : viz_type,
                "NumMentions"  : NumMentions,
                "NumSources":NumSources,
                "coordinates": ast.literal_eval(coordinates),
                "direction":direction,
                "domain":domain,
                "sub_domain":sub_domain,
                "ingested_time": ingested_time,
                "latest":latest,
                "transformed": transformed,
                "value": mean,
                "data":data,
                "description":description,
                "frequency":frequency


            }   
        )
        



In [ ]:
#@title city level final json
final = {
    "features": {
      "properties": li    
    }
    
}

In [ ]:
#@title save city json
with open('/content/drive/MyDrive/all_data/Data/GDELT/JSONS/gdelt_city_top_trends_json.json', 'w') as json_file:
    json.dump(li, json_file)

# Country

In [ ]:
#@title country jsons
data=pd.read_csv('/content/drive/MyDrive/all_data/Data/GDELT/gdelt_all_final_11.csv')
data.drop(['Unnamed: 0','city','state','lat','lon'],axis=1,inplace=True)
data['viz_type'] = 'country level'
data = data.groupby(['Actor1CountryCode','country','name','EventRootCode','viz_type']).agg('sum').reset_index()
data['variable'] = data['EventRootCode'].astype(str).replace(list(code.keys()), list(code.values()))
data['direction'] = data['variable'].replace(list(mapping.keys()), list(mapping.values()))
data.drop(['EventRootCode','NumArticles','country'],axis=1,inplace=True)
data['value'] = ""
data['risk'] = ""

final = pd.DataFrame(columns=list(data.columns))

countries = list(data['Actor1CountryCode'].unique())
for country in countries:
  d1 = data[data['Actor1CountryCode']==country]
  total = d1['NumMentions'].sum()
  d1['value'] = d1['NumMentions']*100/total
  scaler = MinMaxScaler(feature_range=(1, 5))
  value=np.array(d1['value'])
  sc = scaler.fit_transform(value.reshape(-1,1))
  sc= pd.Series(sc.flatten())
  sc1=[]
  for i in sc:
    sc1.append(round(i))
  d1['risk'] = sc1
  final = pd.concat([final,d1])
final.rename(columns={'Actor1CountryCode':'countrycode','name':'country'},inplace=True)
sl = pd.read_csv("/content/projects_final - Sheet1.csv")
po = pd.read_csv("/content/drive/MyDrive/AI Vibrancy 2020/Original data/continents2.xls")
po = po[['alpha-2','alpha-3']]
sl = pd.merge(sl,po,left_on='country',right_on='alpha-2',how='left')
sl.drop(['country','alpha-2'],axis=1,inplace=True)
sl.rename(columns={'alpha-3':'countrycode'},inplace=True)

fd = pd.merge(final,sl,how='left',on='countrycode')
fd = fd.dropna()
fd['coordinates'] = fd['latitude'].astype(str)+","+fd['longitude'].astype(str)
cor = list(fd['coordinates'])
k = []

for c in cor:
  a = []
  c = c.split(',')
  a.append(float(c[0]))
  a.append(float(c[1]))
  k.append(a)
fd['cor'] = k
fd.drop(['coordinates','latitude','longitude'],axis=1,inplace=True)
fd.rename(columns={'cor':'coordinates'},inplace=True)
fd['ingested_time']  = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
fd['latest'] = 'Yes'
fd['transformed'] = 'Yes'
fd['sub_domain'] = fd['direction'].replace([1,-1], ['geopolitical_opportunity','geopolitical_threat'])
fd['domain'] = 'geopolitics'
fd['indicator'] = "Number of mentions"

fd.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_country_2.csv")

In [ ]:
fd=pd.read_csv('/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_country_2.csv')
f1 =fd[fd['countrycode']=="USA"]
f1['value'].sum()

99.99999999999999

In [ ]:
f1

,Unnamed: 0,countrycode,country,viz_type,NumMentions,NumSources,variable,direction,value,risk,coordinates,ingested_time,latest,transformed,sub_domain,domain,indicator
3488,3508,USA,United States,country level,5421920,1552110,Make public statement,1,15.289147,4,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3489,3509,USA,United States,country level,3192293,834503,Appeal,1,9.001874,3,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3490,3510,USA,United States,country level,2286171,569076,Express intent to cooperate,1,6.446721,2,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3491,3511,USA,United States,country level,6982759,1877266,Consult,1,19.690521,5,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3492,3512,USA,United States,country level,2713740,724322,Engage in diplomatic cooperation,1,7.652413,3,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3493,3513,USA,United States,country level,743279,183551,Engage in material cooperation,1,2.095955,1,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3494,3514,USA,United States,country level,1807489,446650,Provide aid,1,5.096897,2,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3495,3515,USA,United States,country level,1600325,410414,Yield,1,4.512720,2,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,1,geopolitics,Number of mentions
3496,3516,USA,United States,country level,894323,212263,Investigate,-1,2.521881,2,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,-1,geopolitics,Number of mentions
3497,3517,USA,United States,country level,470656,125494,Demand,-1,1.327192,1,"[37.09024, -95.712891]",2021-03-12T14:00:50Z,Yes,Yes,-1,geopolitics,Number of mentions


In [ ]:
final = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_country_2.csv")
final.drop('Unnamed: 0',axis=1,inplace=True)
f = final
f = f.sort_values(by=['risk'], ascending=False)
f = f.head(20)
  
f['data'] = "top trends"
f.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_country_2_top_trends.csv")

In [ ]:
#@title country level json
import csv, json
li = []
with open('/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_country_2_top_trends.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    c = 0
    for _, countrycode, country, viz_type, NumMentions, NumSources, variable, direction,value,mean, coordinates, ingested_time,latest,transformed,sub_domain,domain,indicator,data in reader:
        if c==0:
          c = 1
        else:
          li.append({
                "country"  : country,
                "NumMentions(Normalized)": value,
                "variable"   : variable,
                "indicator"  : indicator,
                "countrycode"  : countrycode,
                "viz_type"  : viz_type,
                "NumMentions"  : NumMentions,
                "NumSources"  : NumSources,
                "coordinates": ast.literal_eval(coordinates),
                "direction":direction,
                "domain":domain,
                "sub_domain":sub_domain,
                "ingested_time": ingested_time,
                "latest":latest,
                "transformed": transformed,
                "value":mean,
                "data":data
                }   
          )
          



In [ ]:
#@title country level final json
final = {
    "features": {
      "properties": li    
    }
    
}

In [ ]:
#@title save country json
with open('/content/drive/MyDrive/all_data/Data/GDELT/JSONS/gdelt_country_top_trends_json.json', 'w') as json_file:
    json.dump(li, json_file)

# State

In [ ]:
#@title state jsons
data=pd.read_csv('/content/drive/MyDrive/all_data/Data/GDELT/gdelt_all_final_11.csv')
data.drop(['Unnamed: 0','city','lat','lon'],axis=1,inplace=True)
data['viz_type'] = 'state level'
data = data.groupby(['Actor1CountryCode','country','name','state','EventRootCode','viz_type']).agg('sum').reset_index()
data['variable'] = data['EventRootCode'].astype(str).replace(list(code.keys()), list(code.values()))
data['direction'] = data['variable'].replace(list(mapping.keys()), list(mapping.values()))
data.drop(['EventRootCode','NumArticles','country'],axis=1,inplace=True)
data['value'] = ""
data['risk'] = ""
data = data.dropna()
final = pd.DataFrame(columns=list(data.columns))

countries = list(data['Actor1CountryCode'].unique())
for country in countries:
  d1 = data[data['Actor1CountryCode']==country]
  states = list(d1['state'].unique())
  for state in states:
    d2 = d1[d1['state']==state]
    total = d2['NumMentions'].sum()
    d2['value'] = d2['NumMentions']*100/total
    scaler = MinMaxScaler(feature_range=(1, 5))
    value=np.array(d2['value'])
    sc = scaler.fit_transform(value.reshape(-1,1))
    sc= pd.Series(sc.flatten())
    sc1=[]
    for i in sc:
      sc1.append(round(i))
    d2['risk'] = sc1
    final = pd.concat([final,d2])

final.rename(columns={'Actor1CountryCode':'countrycode','name':'country'},inplace=True)
sl = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/gdelt_state_latlon1.csv")
fd = pd.merge(final,sl,how='left',on=['country','state'])
fd = fd.dropna()
fd['coordinates'] = fd['latitude'].astype(str)+","+fd['longitude'].astype(str)
cor = list(fd['coordinates'])
k = []

for c in cor:
  a = []
  c = c.split(',')
  a.append(float(c[0]))
  a.append(float(c[1]))
  k.append(a)
fd['cor'] = k
fd.drop(['coordinates','latitude','longitude'],axis=1,inplace=True)
fd.rename(columns={'cor':'coordinates'},inplace=True)
fd = fd.sort_values(by=['value'], ascending=False)
fd['ingested_time']  = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
fd['latest'] = 'Yes'
fd['transformed'] = 'Yes'
fd['sub_domain'] = fd['direction'].replace([1,-1], ['geopolitical_opportunity','geopolitical_threat'])
fd['domain'] = 'geopolitics'
fd['indicator'] = "Number of mentions"
fd.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_state_2.csv")


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [ ]:
fd = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_state_2.csv")
f = fd[fd['countrycode']=="USA"]
f = f[f['state']=="Alabama"]
f

,countrycode,country,state,viz_type,NumMentions,NumSources,variable,direction,value,risk,coordinates,ingested_time,latest,transformed,sub_domain,domain,indicator
44887,USA,United States,Alabama,state level,60608,16245,Consult,1,17.140271,5,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions
44884,USA,United States,Alabama,state level,59173,16303,Make public statement,1,16.734446,5,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions
44885,USA,United States,Alabama,state level,31308,8028,Appeal,1,8.854072,3,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions
44900,USA,United States,Alabama,state level,27719,7063,Coerce,-1,7.839084,3,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,-1,geopolitics,Number of mentions
44888,USA,United States,Alabama,state level,27258,7260,Engage in diplomatic cooperation,1,7.708710,3,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions
44902,USA,United States,Alabama,state level,26080,6656,Fight,-1,7.375566,3,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,-1,geopolitics,Number of mentions
44886,USA,United States,Alabama,state level,22346,5358,Express intent to cooperate,1,6.319570,2,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions
44890,USA,United States,Alabama,state level,18266,4382,Provide aid,1,5.165724,2,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions
44894,USA,United States,Alabama,state level,17174,4783,Disapprove,-1,4.856900,2,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,-1,geopolitics,Number of mentions
44891,USA,United States,Alabama,state level,15035,3782,Yield,1,4.251980,2,"[33.258881699999996, -86.8295337]",2021-03-12T14:25:53Z,Yes,Yes,1,geopolitics,Number of mentions


In [ ]:
final = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_state_2.csv")
final.drop('Unnamed: 0',axis=1,inplace=True)
f = pd.DataFrame(columns=list(final.columns))
countries = list(final['country'].unique())
for country in countries:
  d2 = final[final['country']==country]
  d2 = d2.sort_values(by=['risk'], ascending=False)
  d2 = d2.head(3)
  f = pd.concat([f,d2])
  
f['data'] = "top trends"
f.to_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_state_2_top_trends.csv")

In [ ]:
f

,countrycode,country,state,viz_type,NumMentions,NumSources,variable,direction,value,risk,Unnamed: 0.1,coordinates,ingested_time,latest,transformed,sub_domain,domain,indicator,data
607,LVA,Latvia,Lielvarde,state level,44,9,Consult,1,35.772358,5,603.0,"[56.7263693, 24.795892]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
500,LVA,Latvia,Nica,state level,34,3,Consult,1,37.777778,5,502.0,"[56.34578670000001, 21.065709100000007]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
1387,LVA,Latvia,Alsunga,state level,62,10,Make public statement,1,28.181818,5,1340.0,"[56.9795834, 21.5697553]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
57,DMA,Dominica,Saint Luke,state level,15,2,Make public statement,1,88.235294,5,58.0,"[15.4240429, -61.34185329351388]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
4251,DMA,Dominica,Saint Patrick,state level,48,7,Engage in diplomatic cooperation,1,16.382253,5,3039.0,"[15.4240429, -61.34185329351388]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,ZAF,South Africa,Gauteng,state level,24586,7992,Consult,1,20.562530,5,2528.0,"[-25.9363439, 28.0813105]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
3077,ZAF,South Africa,Eastern Cape,state level,4700,1441,Consult,1,19.727177,5,2671.0,"[-32.2171831, 26.6386401]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
3029,SRB,Serbia,Central Serbia,state level,62223,10359,Consult,1,19.928323,5,2632.0,"[43.66464885, 20.990594368313886]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends
4263,SRB,Serbia,Autonomna Pokrajina Vojvodina,state level,856,118,Make public statement,1,16.354605,5,2930.0,"[45.40944260000001, 19.976249203097023]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions,top trends


In [ ]:
 fd = pd.read_csv("/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_state_2.csv")
fd

,Unnamed: 0,countrycode,country,state,viz_type,NumMentions,NumSources,variable,direction,value,risk,Unnamed: 0.1,coordinates,ingested_time,latest,transformed,sub_domain,domain,indicator
0,25197,LVA,Latvia,Cesvaine,state level,8,1,Appeal,1,100.000000,1,18.0,"[56.9657879, 26.3089758]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions
1,10730,DMA,Dominica,Saint Peter,state level,10,1,Provide aid,1,100.000000,1,41.0,"[15.4240429, -61.34185329351388]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions
2,24512,LIE,Liechtenstein,Eschen,state level,6,1,Appeal,1,100.000000,1,24.0,"[47.2126274, 9.5233202]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions
3,33841,PLW,Palau,Ngardmau,state level,4,1,Appeal,1,100.000000,1,47.0,"[7.615, 134.54480767272207]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions
4,33842,PLW,Palau,Ngatpang,state level,10,2,Consult,1,100.000000,1,35.0,"[7.47571625, 134.53330957014066]",2021-03-18T11:36:08Z,Yes,Yes,1,geopolitics,Number of mentions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45817,45799,USA,United States,Virginia,state level,7,2,Use unconventional mass violence,-1,0.000901,1,2868.0,"[37.1232245, -78.4927721]",2021-03-18T11:36:08Z,Yes,Yes,-1,geopolitics,Number of mentions
45818,1459,ARG,Argentina,Cordoba,state level,2,1,Use unconventional mass violence,-1,0.000618,1,1533.0,"[-31.4173391, -64.183319]",2021-03-18T11:36:08Z,Yes,Yes,-1,geopolitics,Number of mentions
45819,45182,USA,United States,Iowa,state level,2,1,Use unconventional mass violence,-1,0.000326,1,2666.0,"[41.9216734, -93.3122705]",2021-03-18T11:36:08Z,Yes,Yes,-1,geopolitics,Number of mentions
45820,45759,USA,United States,Utah,state level,1,1,Use unconventional mass violence,-1,0.000268,1,2627.0,"[39.4225192, -111.714358]",2021-03-18T11:36:08Z,Yes,Yes,-1,geopolitics,Number of mentions


In [ ]:
#@title state level json
import csv, json
li = []
with open('/content/drive/MyDrive/all_data/Data/GDELT/GDELT_MAP_state_2_top_trends.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    c = 0
    for  _,countrycode, country,state, viz_type, NumMentions, NumSources, variable, direction,value,mean,_, coordinates, ingested_time,latest,transformed,sub_domain,domain,indicator,data in reader:
        if c==0:
          c = 1
        else:
          li.append({
                "country"  : country,
                "state" : state,
                "NumMentions(Normalized)"    : value,
                "variable"   : variable,
                "indicator"  : indicator,
                "countrycode"  : countrycode,
                "viz_type"  : viz_type,
                "NumMentions"  : NumMentions,
                "NumSources"  : NumSources,
                "coordinates": ast.literal_eval(coordinates),
                "direction":direction,
                "domain":f"{domain}",
                "sub_domain":sub_domain,
                "ingested_time": ingested_time,
                "latest":latest,
                "transformed": transformed,
                "value":mean,
                "data":data
              }   
          )
        



In [ ]:
#@title state level final json
final = {
    "features": {
      "properties": li    
    }
    
}

In [ ]:
#@title save country json
with open('/content/drive/MyDrive/all_data/Data/GDELT/JSONS/gdelt_state_top_trends_json.json', 'w') as json_file:
    json.dump(li, json_file)